# Fancy but Useful Tensor Operations

In [2]:
import torch
import numpy as np
assert torch.cuda.is_available(), "CUDA is not available"

## 基础语法

### 创建操作

#### 和 NumPy 数组的转换

In [3]:
# 使用 `torch.from_numpy()` (共享内存)
a_array = np.array([1, 2, 3])
a_tensor = torch.from_numpy(a_array)
print("Original PyTorch tensor:", a_tensor)
a_array[0] = 10  # 修改 NumPy 数组
print("Tensor after modifying NumPy array:", a_tensor)

Original PyTorch tensor: tensor([1, 2, 3])
Tensor after modifying NumPy array: tensor([10,  2,  3])


In [4]:
# 使用 `torch.tensor()` (复制内存)
b_array = np.array([[1, 2], [3, 4]])
b_tensor = torch.tensor(b_array)
print("Original PyTorch tensor:")
print(b_tensor)
b_array[0, 0] = 10  # 修改 NumPy 数组
print("Tensor after modifying NumPy array:")
print(b_tensor)

Original PyTorch tensor:
tensor([[1, 2],
        [3, 4]])
Tensor after modifying NumPy array:
tensor([[1, 2],
        [3, 4]])


In [5]:
# 从 Tensor 转换为 NumPy 数组
c_tensor = torch.tensor([1, 2, 3], device='cuda')  # 创建一个在 GPU 上的 Tensor
try: # GPU 上的 Tensor 不能直接转换为 NumPy 数组
    c_array = c_tensor.numpy()
except Exception as e:
    print(e)
c_array = c_tensor.cpu().numpy()
print("NumPy array from CPU tensor:", c_array)

can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.
NumPy array from CPU tensor: [1 2 3]


#### 特殊 Tensor 构造

In [6]:
a = torch.rand(2, 3)
print("Random tensor a:")
print(a)

b = torch.zeros_like(a)
print("\nTensor b with same shape as a:")
print(b)

Random tensor a:
tensor([[0.3439, 0.6145, 0.8293],
        [0.3315, 0.1162, 0.7642]])

Tensor b with same shape as a:
tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [7]:
a = torch.arange(1, 6, 2)
print("Tensor a:", a)

Tensor a: tensor([1, 3, 5])


### 基本属性

In [8]:
a = torch.zeros(2, 3, 4, device='cuda')
print("Rank of a:", a.dim())
print("Shape of a:", a.shape)
print("Shape of dim 1:", a.shape[1])
print("Datatype of a:", a.dtype)
print("Device of a:", a.device)

Rank of a: 3
Shape of a: torch.Size([2, 3, 4])
Shape of dim 1: 3
Datatype of a: torch.float32
Device of a: cuda:0


### 数据类型及设备

In [9]:
x0 = torch.zeros(2, 3, dtype=torch.float16, device='cpu')
print("x0.dtype:", x0.dtype)
print("x0.device:", x0.device)
x1 = x0.float()
print("x1.dtype:", x1.dtype)
x2 = x0.long()
print("x2.dtype:", x2.dtype)
x3 = x0.to(torch.int32)
print("x3.dtype:", x3.dtype)
x4 = x0.to("cuda")
print("x4.device:", x4.device)

x0.dtype: torch.float16
x0.device: cpu
x1.dtype: torch.float32
x2.dtype: torch.int64
x3.dtype: torch.int32
x4.device: cuda:0


## 索引操作

### 单元素索引

In [10]:
a = torch.tensor([[1, 2, 3], [4, 5, 6]])
print("a[0, 1]:", a[0, 1])
print("a[0, 1].item():", a[0, 1].item())

a[0, 1]: tensor(2)
a[0, 1].item(): 2


### 切片索引

In [11]:
a = torch.tensor([10, 11, 12, 13, 14, 15, 16])
print("a[2:5] ", a[2:5])  # Elements between index 2 and 5
print("a[:-1] ", a[:-1])  # All elements except the last one
print("a[::2] ", a[::2])  # Every second element
print("a[:] ", a[:])      # All elements

a[2:5]  tensor([12, 13, 14])
a[:-1]  tensor([10, 11, 12, 13, 14, 15])
a[::2]  tensor([10, 12, 14, 16])
a[:]  tensor([10, 11, 12, 13, 14, 15, 16])


In [12]:
b = torch.tensor(
    [[1, 2, 3, 4],
     [5, 6, 7, 8],
     [9, 10, 11, 12]]
)
# Single row
print("Single row:")
print(b[1, :], b[1, :].shape)  # Equivalent to b[1]
print(b[1:2, :], b[1:2, :].shape)
# Single column
print("\nSingle column:")
print(b[:, 2], b[:, 2].shape)
print(b[:, 2:3], b[:, 2:3].shape)
# All columns except the last one
print("\nAll columns except the last one:")
print(b[:, :-1], b[:, :-1].shape)

Single row:
tensor([5, 6, 7, 8]) torch.Size([4])
tensor([[5, 6, 7, 8]]) torch.Size([1, 4])

Single column:
tensor([ 3,  7, 11]) torch.Size([3])
tensor([[ 3],
        [ 7],
        [11]]) torch.Size([3, 1])

All columns except the last one:
tensor([[ 1,  2,  3],
        [ 5,  6,  7],
        [ 9, 10, 11]]) torch.Size([3, 3])


In [13]:
c = torch.zeros(2, 4, dtype=torch.int64)
c[:, :2] = 1
c[:, 2:] = torch.tensor([[2, 3], [4, 5]])
print(c)

tensor([[1, 1, 2, 3],
        [1, 1, 4, 5]])


### 整数 Tensor 索引

In [15]:
a = torch.arange(12).reshape(3, 4)
print("Original tensor a:")
print(a)

idx = torch.tensor([0, 0, 2, 1, 1])
print('\nReordered rows:')
print(a[idx])

idx = torch.tensor([3, 2, 1, 0])
print('\nReordered columns:')
print(a[:, idx])

Original tensor a:
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

Reordered rows:
tensor([[ 0,  1,  2,  3],
        [ 0,  1,  2,  3],
        [ 8,  9, 10, 11],
        [ 4,  5,  6,  7],
        [ 4,  5,  6,  7]])

Reordered columns:
tensor([[ 3,  2,  1,  0],
        [ 7,  6,  5,  4],
        [11, 10,  9,  8]])
